In [5]:
import numpy as np
import cvxpy as cp
import pandas as pd
import mosek
import matplotlib.pyplot as plt
import Hit_and_Run as hr
import phi_divergence as phi
from scipy.optimize import fsolve
import scipy.stats as stats 
import datetime as date
from datetime import datetime as dt
from dateutil.relativedelta import *

In [35]:
def poly_cvar_port_pos(p,alpha,R,f,l,W0):
    n = len(R)
    I = len(R[0])
    a = cp.Variable(I)
    theta_1 = cp.Variable(1)
    theta_2 = cp.Variable(1)
    t = cp.Variable(n)
    z = cp.Variable(n, nonneg= True)
    s = cp.Variable(n)
    constraints = [cp.sum(a)==1, a>=0, 1-s/l*(1-p) <= z]
    X = W0*R @ a
    som = 0
    for i in range(n):
        constraints.append(theta_1 + cp.pos(1/alpha*(theta_2-X[i])) <= s[i])
        arg = cp.pos(1+s[i]/l*(p-1))
        constraints.append(l*(1/p*(arg)**(p/(p-1))-1/p) <= t[i])
    obj = cp.Minimize(-theta_1-theta_2+f@t)
    prob = cp.Problem(obj,constraints)
    prob.solve(solver = cp.MOSEK)
    return(prob.value, a.value)

def poly_cvar_port_ball(p,alpha,R,f,r,W0):
    n = len(R)
    I = len(R[0])
    a = cp.Variable(I)
    theta_1 = cp.Variable(1)
    theta_2 = cp.Variable(1)
    lbda = cp.Variable(1, nonneg = True)
    t = cp.Variable(n)
    z = cp.Variable(n, nonneg= True)
    s = cp.Variable(n)
    X = W0*R @ a
    constraints = [lbda-s*(1-p) <= z, cp.sum(a)==1, a>= 0]
    macht = np.array([(p-1)/p, 1-(p-1)/p])
    for i in range(n):
        constraints.append(theta_1 + cp.pos(1/alpha*(theta_2-X[i])) <= s[i])
        constraints.append(z[i] <= cp.geo_mean(cp.vstack([lbda+p*t[i],lbda]),macht))
    obj = cp.Minimize(-theta_1-theta_2+lbda*r+t@f)
    prob = cp.Problem(obj,constraints)
    prob.solve(solver = cp.MOSEK)
    return(prob.value, a.value)


def cvar_port(alpha,R,f):
    n = len(R)
    I = len(R[0])
    a = cp.Variable(I)
    eta = cp.Variable(1)
    som = 0
    constraints = [cp.sum(a)==1, a>= 0]
    X = R @ a
    for i in range(n):
        som = som - f[i]*cp.pos(-1/alpha * (X[i]-eta))
    obj = cp.Maximize(eta + som)
    prob = cp.Problem(obj,constraints)
    prob.solve(solver = cp.MOSEK)
    return(prob.value, a.value)

def poly_cvar(p,alpha,x,f,l):
    n = len(x)
    theta_1 = cp.Variable(1)
    theta_2 = cp.Variable(1)
    t = cp.Variable(1)
    z = cp.Variable(n, nonneg= True)
    s = cp.Variable(n)
    constraints = [1-s/l*(1-p) <= z]
    som = 0
    for i in range(n):
        constraints.append(theta_1 + cp.pos(1/alpha*(theta_2-x[i])) <= s[i])
        som = som + l*f[i]*(1/p*(z[i]**(p/(p-1)))-1/p)
    constraints.append(som <= t)
    obj = cp.Maximize(theta_1+theta_2-t)
    prob = cp.Problem(obj,constraints)
    prob.solve(solver = cp.MOSEK)
    return(prob.value, theta_1.value, theta_2.value)

def cvar_fast(alpha,x,f):
    n = len(f)
    order = np.argsort(x)
    x = np.sort(x)
    f = f[order]
    csum = np.cumsum(f)
    if csum[0] > alpha:
        return(x[0])
    else:
        k_max = np.max(np.where((csum < alpha)==True))
    Ex = np.sum(np.multiply(x[0:k_max+1],f[0:k_max+1]))
    last = (alpha - csum[k_max])*x[k_max+1]
    return(1/alpha*(Ex+last))

def poly_cvar(p,alpha,x,f,l):
    n = len(x)
    theta_1 = cp.Variable(1)
    theta_2 = cp.Variable(1)
    t = cp.Variable(1)
    z = cp.Variable(n, nonneg= True)
    s = cp.Variable(n)
    constraints = [1-s/l*(1-p) <= z]
    som = 0
    for i in range(n):
        constraints.append(theta_1 + cp.pos(1/alpha*(theta_2-x[i])) <= s[i])
        som = som + l*f[i]*(1/p*(z[i]**(p/(p-1)))-1/p)
    constraints.append(som <= t)
    obj = cp.Maximize(theta_1+theta_2-t)
    prob = cp.Problem(obj,constraints)
    prob.solve(solver = cp.MOSEK)
    return(prob.value, theta_1.value, theta_2.value)

In [30]:
df_returns6 = pd.read_csv('6_Portfolios_2x3.csv', skiprows = 15) 
df_returns = df_returns6[0:1144].copy()
df_returns['Date'] = pd.to_datetime(df_returns['Date'], format = '%Y%m')
for i in range(1, len(df_returns.columns)):
    df_returns[df_returns.columns[i]] = pd.to_numeric(df_returns[df_returns.columns[i]])
df_returns

,Date,SMALL LoBM,ME1 BM2,SMALL HiBM,BIG LoBM,ME2 BM2,BIG HiBM
0,1926-07-01,1.0874,0.9349,-0.0695,5.7168,1.9620,1.4222
1,1926-08-01,0.7030,1.2300,5.3842,2.7154,2.6930,6.3154
2,1926-09-01,-2.9117,-0.1303,-0.4374,1.4287,0.0704,-0.7967
3,1926-10-01,-3.8196,-4.5860,-2.0112,-3.5898,-2.3398,-4.0970
4,1926-11-01,3.1806,3.7233,2.0944,3.1292,2.8952,3.4614
...,...,...,...,...,...,...,...
1139,2021-06-01,5.6058,0.4400,-1.0979,4.8188,-1.2594,-4.0036
1140,2021-07-01,-5.5593,-1.8623,-3.6521,3.1048,-0.0099,-2.3000
1141,2021-08-01,2.3903,1.5124,2.6680,3.5667,1.4122,3.0371
1142,2021-09-01,-4.3421,-3.4661,0.6445,-5.4525,-3.8570,-0.2526


In [31]:
startdate = dt(1980,1,3)
enddate = dt(2014,1,3)
X = df_returns[np.logical_and(df_returns.Date >= startdate, df_returns.Date <= enddate )][df_returns.columns[1:7]]
X = X.reset_index(drop = True)
R = X.to_numpy()     
R = R/100
N = len(X)
r = 1/(2*N)*stats.chi2.ppf(0.95, N-1)   
#### The risk free rate is taken to be 0.07%
r_f = 0.032 
W0 = 10000
p = 2
f = np.zeros(N)+1/N
alpha = 0.05

In [32]:
poly_cvar_port_ball(p,alpha,R,f,r,W0)

(1978.8842353810314,
 array([1.04553304e-11, 1.95359910e-11, 1.82151930e-11, 2.08959622e-09,
        4.13680611e-01, 5.86319387e-01]))

In [33]:
poly_cvar_port_ball(p,alpha,R,f,0,W0)

(961.7714444923784,
 array([3.31617712e-07, 1.11462035e-06, 6.59958283e-07, 5.22263451e-01,
        4.77733119e-01, 1.32391413e-06]))

In [45]:
poly_cvar_port_pos(p,alpha,R,f,1,W0)

(1978.5577127784168,
 array([2.33367348e-10, 3.77938849e-10, 3.73095456e-10, 1.22302667e-09,
        4.14982819e-01, 5.85017178e-01]))

array([[-0.003848, -0.029529, -0.026733, -0.03063 ,  0.021593,  0.00445 ],
       [-0.193301, -0.1604  , -0.173869, -0.090773, -0.10717 , -0.13036 ],
       [ 0.06233 ,  0.064092,  0.060035,  0.037499,  0.055976,  0.061397],
       ...,
       [ 0.052722,  0.038996,  0.045033,  0.033407,  0.018158,  0.046605],
       [ 0.01859 ,  0.028208,  0.018819,  0.021247,  0.03716 ,  0.020913],
       [-0.001778, -0.040237, -0.036582, -0.033322, -0.033827, -0.039001]])